---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [7]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    txt_file_data =  pd.read_table('university_towns.txt', header=None)
    
    txt_file_data.columns = ['StateRegion']
    
    txt_file_data['IsState'] = txt_file_data['StateRegion'].str.contains(r'\[edit\]')
    
    txt_file_data['StateRegion'] = txt_file_data['StateRegion'].str.replace(r'\[.*\]', '')
    
    txt_file_data['StateRegion'] = txt_file_data['StateRegion'].str.replace(r'\s\(.*', '')
    
    states_series = txt_file_data['StateRegion'].where(txt_file_data['IsState']).dropna().reset_index()['StateRegion']
    
    is_state = txt_file_data['IsState']
    
    j = -1
    
    state_list = []
    
    for state_flag in is_state.values:
        if state_flag:
            j+=1
        else:
            state_list.append(states_series[j])
    
    state_df = pd.DataFrame(state_list, columns=['State'])
    
    region_df = txt_file_data['StateRegion'].where(txt_file_data['IsState'] == False).dropna().reset_index()[['StateRegion']]
    
    region_df.columns = ['RegionName']
    
    university_towns_df = pd.merge(state_df, region_df, how='inner', left_index=True, right_index=True)
    
    return university_towns_df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    GDP_lev = pd.read_excel('gdplev.xls', header=None, footer=None)[[4, 6]]
    
    GDP_lev = GDP_lev.where(GDP_lev[4]>='2000q1').dropna().reset_index()[[4, 6]]
    
    GDP_lev.columns = ['Quarters', 'GDP in billions of chained 2009 dollars']
    
    GDP_lev = GDP_lev.set_index('Quarters')
    
    GDP_lev['Rise(R)/Fall(F)'] = (GDP_lev['GDP in billions of chained 2009 dollars'] > GDP_lev['GDP in billions of chained 2009 dollars'].shift()).replace({True:'R', False:'F'})
    
    GDP_lev['Recession'] = (GDP_lev['Rise(R)/Fall(F)'] == 'F') & (GDP_lev['Rise(R)/Fall(F)'].shift(-1) == 'F')
    
    return GDP_lev.where(GDP_lev['Recession'] == True).dropna().head().index[0]

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    GDP_lev = pd.read_excel('gdplev.xls', header=None, footer=None)[[4, 6]]
    
    GDP_lev = GDP_lev.where(GDP_lev[4]>='2000q1').dropna().reset_index()[[4, 6]]
    
    GDP_lev.columns = ['Quarters', 'GDP in billions of chained 2009 dollars']
    
    GDP_lev = GDP_lev.set_index('Quarters')
    
    GDP_lev['Rise(R)/Fall(F)'] = (GDP_lev['GDP in billions of chained 2009 dollars'] > GDP_lev['GDP in billions of chained 2009 dollars'].shift()).replace({True:'R', False:'F'})
    
    GDP_lev['Recession'] = (GDP_lev['Rise(R)/Fall(F)'] == 'F') & (GDP_lev['Rise(R)/Fall(F)'].shift(-1) == 'F')
    
    GDP_lev['Upturn'] = (GDP_lev['Rise(R)/Fall(F)'] == 'R') & (GDP_lev['Rise(R)/Fall(F)'].shift() == 'R')
    
    Recession_starts = GDP_lev.where(GDP_lev['Recession'] == True).dropna().head().index[0]
    
    return GDP_lev[Recession_starts:].where(GDP_lev['Upturn'] == True).dropna().head().index[0]

get_recession_end()

'2009q4'

In [11]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    GDP_lev = pd.read_excel('gdplev.xls', header=None, footer=None)[[4, 6]]
    
    GDP_lev = GDP_lev.where(GDP_lev[4]>='2000q1').dropna().reset_index()[[4, 6]]
    
    GDP_lev.columns = ['Quarters', 'GDP in billions of chained 2009 dollars']
    
    GDP_lev = GDP_lev.set_index('Quarters')
    
    GDP_lev['Rise(R)/Fall(F)'] = (GDP_lev['GDP in billions of chained 2009 dollars'] > GDP_lev['GDP in billions of chained 2009 dollars'].shift()).replace({True:'R', False:'F'})
    
    GDP_lev['Recession'] = (GDP_lev['Rise(R)/Fall(F)'] == 'F') & (GDP_lev['Rise(R)/Fall(F)'].shift(-1) == 'F')
    
    GDP_lev['Upturn'] = (GDP_lev['Rise(R)/Fall(F)'] == 'R') & (GDP_lev['Rise(R)/Fall(F)'].shift() == 'R')
    
    Recession_starts = GDP_lev.where(GDP_lev['Recession'] == True).dropna().head().index[0]
    
    Recession_ends = GDP_lev[Recession_starts:].where(GDP_lev['Upturn'] == True).dropna().head().index[0]

    return GDP_lev[Recession_starts : Recession_ends]['GDP in billions of chained 2009 dollars'].astype('float64').idxmin()

get_recession_bottom()

'2009q2'

In [13]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    Housing_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    Housing_df['State'] = Housing_df['State'].replace(states)
    
    Housing_df = (Housing_df.sort_values('State')).set_index(['State', 'RegionName'])
    
    Housing_df = Housing_df[Housing_df.filter(regex='^2').columns]
    
    Housing_df = Housing_df.groupby(pd.PeriodIndex(Housing_df.columns, freq='q'), axis=1).mean().rename(columns=lambda c: str(c).lower())

    return Housing_df

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   164400.000000  165900.000000  167600.000000   
        Satsuma       88600.000000   90966.666667   90333.333333   
        Irvington     72466.666667   74100.000000   75700.000000   
        Mount Olive  106500.000000  107333.333333  106966.666667   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Bar Nunn      93033.333333   89666.666667   88900.000000   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   169766.666667  172533.333333  173433.333333   
        Satsuma       89600.000000   90233.333333   90766.666667   
        Irvington     76266.666667   76300.000000   74300.000000   
        Mount Olive  107200.000000  108266.666667  108766.666667   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Bar Nunn      93500.000000   98833.333333   99900.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   174166.666667  176700.000000  178000.000000   
        Satsuma       90633.333333   92200.000000   93000.000000   
        Irvington     73500.000000   73833.333333   75966.666667   
        Mount Olive  110166.666667  112400.000000  113966.666667   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Bar Nunn      99100.000000   98333.333333   98633.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Mulga                  NaN  ...   45466.666667   43600.000000   
        Trussville   180800.000000  ...  212033.333333  210966.666667   
        Satsuma       93200.000000  ...  125266.666667  127500.000000   
        Irvington     79266.666667  ...   90533.333333   95666.666667   
        Mount Olive  115000.000000  ...  132066.666667  132366.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Bar Nunn     100866.666667  ...  199233.333333  200933.333333   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Mulga         42166.666667   41933.333333   42733.333333   
        Trussville   214566.666667  216600.000000  220066.666667   
        Satsuma      128266.666667  129266.666667  131300.000000   
        Irvington     98466.666667   99500.000000   98800.000000   
        Mount O

In [15]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    recession_start = get_recession_start()
    
    recession_bottom = get_recession_bottom()
    
    Housing_df = convert_housing_data_to_quarters()
    
    columns_to_keep = Housing_df.columns.tolist() 
    
    recession_start_index = columns_to_keep.index(recession_start)
    
    recession_bottom_index = columns_to_keep.index(recession_bottom)
    
    columns_to_keep = columns_to_keep[recession_start_index-1 : recession_bottom_index+1]
    
    quarter_before_recession = columns_to_keep[0]
    
    Housing_df = Housing_df[columns_to_keep]
    
    university_town_list = get_list_of_university_towns() 
    
    university_town_list['IsUnivTwn'] = True
    
    university_town_list = (university_town_list.sort_values('State')).set_index(['State', 'RegionName'])
    
    df = pd.merge(Housing_df, university_town_list, how='outer', left_index=True, right_index=True)
    
    df['IsUnivTwn'] = df['IsUnivTwn'].replace({np.NaN: False})
    
    df['PriceRatio'] = df[quarter_before_recession]/df[recession_bottom]
    
    university_towns = df[df['IsUnivTwn'] == True]['PriceRatio']
    
    non_university_towns = df[df['IsUnivTwn'] == False]['PriceRatio']
     
    ttest_result = ttest_ind(university_towns, non_university_towns, nan_policy='omit')   
    
    pvalue = ttest_result[1]
    
    mean_pr_univ = university_towns.mean()
    
    mean_pr_non_univ = non_university_towns.mean()
    
    if pvalue < 0.01:
        different = True
    else:
        different = False
    
    if mean_pr_univ < mean_pr_non_univ:
        better = 'university town'
    else:
        better = 'non-university town'
    
    return (different, pvalue, better)

run_ttest()

(True, 0.002724063704753125, 'university town')